# Implementação didática do RSA com Python puro

In [1]:
# Etapa 1: Escolher dois primos pequenos
# p1 = 54654656477
# q1 = 34654655543
# p2 = 34654656007
# q2 = 123412353121
# p3 = 123412353199
# q3 = 213462437
# n1 = p1 * q1
# n2 = p2 * q2
# n3 = p3 * q3

p = 23
q = 17
p = 54654656477
q = 34654655543
n = p * q

print("n =", n)

n = 1894038294031428902011


In [2]:
# Etapa 2: Calcular a função totiente de Euler φ(n)
phi = (p - 1) * (q - 1)
print("phi(n) =", phi)

phi(n) = 1894038293942119589992


In [3]:
# Etapa 3: Escolher e tal que 1 < e < phi e gcd(e, phi) = 1

def mdc(a, b):
    while b != 0:
        a, b = b, a % b
    return a

e = 3  # 65537
print("gcd(e, phi):", mdc(e, phi))

gcd(e, phi): 1


In [4]:
# Etapa 4: Calcular o inverso modular d de e mod phi usando o algoritmo de Euclides estendido

def egcd(a, b):
    if b == 0:
        return (1, 0)
    else:
        x, y = egcd(b, a % b)
        return (y, x - (a // b) * y)

def modinv(e, phi):
    x, _ = egcd(e, phi)
    return x % phi

d = modinv(e, phi)
print("d =", d)

d = 1262692195961413059995


In [5]:
# Etapa 5: Gerar as chaves pública e privada
public_key = (e, n)
private_key = (d, n)
print("Chave pública:", public_key)
print("Chave privada:", private_key)

Chave pública: (3, 1894038294031428902011)
Chave privada: (1262692195961413059995, 1894038294031428902011)


In [6]:
# Etapa 6: Cifrar uma mensagem M com a chave pública
def message_to_int(m):
   mb= m.encode('utf-8')
   mi= int.from_bytes(mb,byteorder='big')
   return mi
M =  message_to_int("SEGREDO") # Mensagem original (deve ser < n)
assert M < n
print("M em int:", M)
C = pow(M, e, n)
print("Mensagem criptografada:", C)

M em int: 23438595692250191
Mensagem criptografada: 374038228698701576379


In [7]:
# Etapa 7: Decifrar com a chave privada
def int_to_message(m):
    return m.to_bytes((m.bit_length() + 7) // 8, byteorder='big').decode('utf-8')
decrypted = pow(C, d, n)
print("Mensagem decriptada:", decrypted, "->",int_to_message(decrypted))
# Verificação final
assert M == decrypted
print("✅ Mensagem decifrada corretamente!")

Mensagem decriptada: 23438595692250191 -> SEGREDO
✅ Mensagem decifrada corretamente!


In [14]:
# 🔓 Tentativa de quebra 1: fatorar n para encontrar p e q

def fatorar_n(n):
    for i in range(2, n):
        if n % i == 0:
            return i, n // i
    return None, None

fp, fq = fatorar_n(n)
print("Fatores encontrados:", fp, fq)
print("n=",fp*fq)

KeyboardInterrupt: 

In [36]:
# 🔓 Tentativa de quebra 2: logaritmo discreto (força bruta)

def log_discreto(C, e, n):
    for m in range(n):
        if pow(m, e, n) == C:
            return m
    return None

brute_force_message = log_discreto(C, e, n)
print("Mensagem recuperada via força bruta:", brute_force_message, "->",int_to_message(brute_force_message))

KeyboardInterrupt: 

In [8]:
# 🔓 Tentativa de ataque com broadcast 3


from sympy import integer_nthroot
from sympy.ntheory.modular import crt

# Função para raiz cúbica inteira
def raiz_cubica_inteira(x):
    raiz, exato = integer_nthroot(x, 3)
    assert exato, "Raiz cúbica não exata!"
    return raiz

# Usando Ns distintos
p1 = 54654656477
q1 = 34654655543
p2 = 34654656007
q2 = 123412353121
p3 = 123412353199
q3 = 213462437
n1 = p1 * q1
n2 = p2 * q2
n3 = p3 * q3

# Aplicar Teorema Chinês do Resto para recuperar m^e
c1 = pow(M, e, n1)
c2 = pow(M, e, n2)
c3 = pow(M, e, n3)
modulos = [n1, n2, n3]
restos = [c1, c2, c3]
m_e, _ = crt(modulos, restos)

print("m^e reconstruído (sem mod):", m_e)

# Extrair raiz cúbica inteira
m_recover = raiz_cubica_inteira(m_e)
mensagem_recover = m_recover.to_bytes((m_recover.bit_length() + 7) // 8, 'big')
print("Mensagem recuperada:", mensagem_recover.decode())

m^e reconstruído (sem mod): 12876409001086457831994553259920550719159423717871
Mensagem recuperada: SEGREDO
